In [21]:
import numpy as np
X = np.random.randint(5, size=(6, 100))
y = np.array([1, 2, 3, 4, 5, 6])
from sklearn.naive_bayes import MultinomialNB, GaussianNB
clf =  MultinomialNB()
clf.fit(X, y)

test = X[0]
test[1] = 0
test = np.random.randint(5, size=(100))

print(clf.predict_proba(test))

[[  7.94771063e-06   9.90030296e-01   9.86937379e-03   5.95542608e-08
    7.37611133e-05   1.85614682e-05]]


In [2]:
### normcdf
import numpy as np
from scipy.special import erfc
import math

def normcdfM(x, m=None, s=None):
    if m==None and s==None:
        z = x
    elif s==None:
        z = x-m
    else:
        z = (x-m)/s
        
    return 0.5*erfc(-z/math.sqrt(2))
    

In [3]:
### normpdf

import numpy as np
import math

def normpdfM(x, m=0, s=1):    
    o = 1.0/math.sqrt(2.0*math.pi*(s**2))
    
    return o*math.exp((-((x-m)**2)/(2*(s**2)))[0])
    

In [38]:
##tmean

import numpy as np

def tmean(m, index_max, n_samps):
    
    K = m.shape[0]
    u = np.random.randn(n_samps,1)

    t = m[index_max]*np.ones((K, 1)) - m 
    tr = t
    t = np.delete(t, index_max, 0)
    
    s = np.tile(u, (1, K-1)) + np.tile(t,(1, n_samps)).conj().T
    z = np.mean(np.prod(safenormcdf(s.conj().T), 0))

    tm = np.zeros(m.shape[0])
    for r in range(0, K):
        sr = np.tile(u, (1, K)) + np.tile(tr, (1, n_samps)).conj().T
        sr.take([r, index_max], axis=1) # TEST
        
        nr = np.mean(safenormpdf(u.conj().T + m[index_max] 
                                 - m[r])*np.prod(safenormcdf(sr.conj().T), 1))
        
        if r == index_max:
            tm[r] = 0
        else:
            tm[r] = m[r] - nr/z
            
    tm[index_max] = np.sum(m, axis=0) - np.sum(tm, axis=0)
    tm = tm.conj().T                    
            
    return tm, z
                                                               

def safenormcdf(x):
    #x = x[0]
    thresh=-10;
    x[np.nonzero(x<thresh)] = thresh
    return normcdfM(x)

def safenormpdf(x):
    x=x[0]
    thresh=35;
    x[np.nonzero(x<-thresh)] = -thresh
    x[np.nonzero(x>thresh)] = thresh
    return normpdfM(x)

def safelogsc(x):
    if x<1e-300:
        x = 1e-200
    elif x>1e300:
        x = 1e300
    return math.log(x)

def safelog(x):
    x[np.nonzero(x<1e-300)] = 1e-200
    x[np.nonzero(x>1e300)] = 1e300
    return math.log(x)
        

In [218]:
import numpy as np

def dist(X,Y,Z): # X, Y, Z matrices
    
    nx = X.shape[0]
    ny = Y.shape[0]
    
    #vector = np.sum((X**2)*Z, 1).conj().T
    #vector = vector.reshape(1, vector.shape[0])
    #print np.shape(np.ones((nx,1))*vector)
    #distance= (np.sum((X**2)*Z, 1)*np.ones((1,ny))+
    #       np.ones((nx,1))*vector - 2*(np.dot(X*Z, Y.conj().T)))

    distance= (np.sum((X**2)*Z, 1)*np.ones((1,ny))+
              np.ones((nx,1))*(np.sum((X**2)*Z, 1).conj().T) - 2*(np.dot(X*Z, Y.conj().T))) ## NOT WORKING
           
    return distance


def VarMultProbRegGP(X, t, X_test=0, t_test=0, theta=0, theta_estimate=0, 
                     nos_its=0, kernel_Type=0, poly_kernel_power=0, thresh=0):

    SMALL_NOS = 1e-10

    nos_samps_tg = 1000

    nos_samps_is = 1000

    sigma = 1e-6

    tau = 1e-6

    C = np.unique(t).size
    N, D = X.shape
    Y = np.random.randn(N, C)
    M = np.random.rand(N, C)
    beta = theta
    Theta = np.diag(theta)

    psi = np.ones((1, theta.size)) 
    In = np.eye(N)
    Ic = np.eye(C)

    diff = 1e100
    its = 0

    #K = np.exp(-dist(X,X,Theta)) + np.eye(N)*SMALL_NOS; # TODO: implement create kernel function
    K = np.dot(np.dot(X, Theta), X.conj().T) + np.eye(N)*SMALL_NOS
    
    iK = np.linalg.inv(K+In)
    Ki = np.dot(K, iK)
    
    #THETA = []
    LOWER_BOUND = [-1e-3]
    #PL = []
    
    test_err = []
    while its<nos_its and diff>thresh:
        its += 1
        
        for k in range(0, C):
            M[:,[k]] = np.dot(Ki, Y[:,[k]])
            #M[:,k] = Ki*Y[:,[k]]

        lower_bound = 0    
        for n in range(0, N):
            a, z = tmean(M[[n],].conj().T, t[n], nos_samps_tg)
            Y[n] = a
            lower_bound = lower_bound + safelogsc(z)
         
        if its == 2: 
             LOWER_BOUND[0] = LOWER_BOUND[1] #### WHY? 
        
        lower_bound = (lower_bound-0.5*C*np.trace(np.dot(K, iK))
                                  -0.5*np.sum(np.diag(np.dot(np.dot(M.conj().T, iK), M)))
                                  -0.5*C*np.trace(iK)
                                  -0.5*C*safelogsc(abs(np.linalg.det(K)))
                                  +0.5*C*safelogsc(abs(np.linalg.det(np.dot(K, iK))))
                                  -0.5*N*C*safelogsc(2*math.pi)
                                  +0.5*N*C + 0.5*N*safelogsc(2*math.pi))
        
        LOWER_BOUND.append([lower_bound]) 
        if its == 2: 
             LOWER_BOUND[0] = LOWER_BOUND[1]                                            
                                                    
        diff = abs(100*(lower_bound - LOWER_BOUND[-2])/LOWER_BOUND[-2])
        print its
        
    ####### predict ########

    n_test = X_test.shape[0]
    #K_test = np.exp(-dist(X,X_test,Theta))
    #K_test_self = np.exp(-dist(X_test,X_test,Theta))
    K_test = np.dot(np.dot(X, Theta), X_test.conj().T)
    K_test_self = np.dot(np.dot(X_test, Theta), X_test.conj().T)

    S = (np.diag(K_test_self) - np.diag(np.dot(np.dot(K_test.conj().T, iK), K_test))).conj().T

    predictive_likelihood = 0
    res = (np.dot(np.dot(Y.conj().T, iK), K_test)).conj().T
    P_test = np.ones((n_test, C))
    u = np.random.randn(nos_samps_tg, 1)

    for n in range(0, n_test):
        for i in range(0, C):
            pp = np.ones((nos_samps_tg, 1))
            for j in range(0, C):
                if j!=i:
                    pp = pp*safenormcdf(u+(res[n,i]-res[n,j])/(math.sqrt(1+S[n])))

            P_test[n,i] = np.mean(pp)
        P_test[n] = P_test[n]/np.sum(P_test[n])

        #predictive_likelihood = predictive_likelihood + safelog(P_test[n, t_test[n]])
        
    
    print P_test
        
    
# X_1 = np.random.rand(20, 5)
# X_2 = 100*np.random.rand(20,5)
# X = np.concatenate((X_1, X_2))
# X_test = np.random.rand(50,5)

# y = np.array([0 for x in range(0, 20)]+[1 for x in range(0, 20)])
# theta = np.random.rand(5)
# X = np.array([[1.0, 1.0],[2.0, 2.0],[3.0, 3.0]])
# y = np.array([0, 0, 1]) #starting from 0
# X_test = np.array([[1.0, 2.0]])
# theta = np.array([0.169, 0.334])

X, t = generate(500)
X_t, t_t = generate(5000)
theta = np.random.rand(10)

VarMultProbRegGP(X, t, theta=theta, nos_its=50, thresh = 0.000001, X_test=X_t)    



1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
[[ 0.35300665  0.34063847  0.30635489]
 [ 0.40794313  0.3394463   0.25261057]
 [ 0.35743945  0.33442413  0.30813641]
 ..., 
 [ 0.28321715  0.34859617  0.36818668]
 [ 0.3359284   0.30741718  0.35665442]
 [ 0.32976065  0.33799741  0.33224194]]


In [213]:
def generate(n):

    u = 4*np.random.rand(n,2)-2    
    
    i = np.nonzero( (u[:,0]**2 + u[:,1]**2 > 0.1) & (u[:,0]**2 + u[:,1]**2 < 0.5) )
    j = np.nonzero( (u[:,0]**2 + u[:,1]**2 > .6) & (u[:,0]**2 + u[:,1]**2 < 1) )
    X = u[np.concatenate((i[0],j[0])),:]

    t = np.zeros((i[0].shape[0],1), dtype=int)
    t = np.concatenate((t, np.ones((j[0].shape[0],1), dtype=int)))
    x = 0.1*np.random.randn(i[0].shape[0],2)
    
    k = np.nonzero(x[:,0]**2 + x[:,1]**2 < 0.1)
    X = np.concatenate((X, x[k[0],:]))
    t = np.concatenate((t, 2*np.ones((k[0].shape[0],1), dtype=int)))
    X = np.concatenate((X, np.random.randn(X.shape[0],8)), 1)

    
    return X, t


In [196]:
A = np.array([[1,2],[3,4]])
B = np.array([[5,6],[7,8]])

print np.concatenate((A, B))

[[1 2]
 [3 4]
 [5 6]
 [7 8]]
